# Exchange Rates ETL Pipeline

In [ ]:
import requests
import pandas as pd
import mysql.connector
from mysql.connector import connect
from datetime import datetime

## Extract data from ExchangeRate.host

In [ ]:
base_currency = 'USD'
url = f"https://api.frankfurter.app/latest?from={base_currency}"

response = requests.get(url)
data = response.json()

In [ ]:
#USe the APIs date 

timestamp = datetime.strptime(data.get('date', datetime.now().strftime("%Y-%m-%d")), "%Y-%m-%d")
rates = data.get('rates', {})

In [ ]:
#Create DataFrame

df_rates = pd.DataFrame(rates.items(), columns=["target_currency", "rate"])
df_rates["base_currency"] = base_currency
df_rates["timestamp"] = timestamp

print("\nSample Extracted data : ")
print(df_rates.head())

## Load into MYSQL

In [ ]:
# Connect to SQL
try:
    conn = mysql.connector.connect(
        host = 'localhost',
        user = 'root',
        password = 'your_password',
        database = 'your_database_name'
    )

    cursor = conn.cursor()
    print("Connected to SQL")
except Error as e:
    print("Error:", e)

In [ ]:
#Create table if not exists.

cursor.execute("""
    create table if not exists exchange_rates(
	id int auto_increment primary key,
    base_currency varchar(10),
    target_currency varchar(10),
    rate float,
    timestamp datetime,
    unique (base_currency, target_currency, timestamp)
)
""")

In [ ]:
#Inserting Values in the database

for _, row in df_rates.iterrows():
    cursor.execute("""
        insert ignore into exchange_rates(base_currency, target_currency, rate, timestamp)
        values(%s, %s, %s, %s)
    """,(
        row['base_currency'],
        row['target_currency'],
        row['rate'],
        row['timestamp']
    ))

conn.commit()
print("\nData Inserted into MySQL table : exchange_rates")

## Query & Analysis

In [ ]:
# Query 1 : Highest Exchange Rate

df_highest = pd.read_sql("""
    Select target_currency, rate
    From exchange_rates
    Where base_currency = 'USD'
    Order By rate DESC
    Limit 10
""",conn)
print("\n Highest Exchange Rate: ")
print(df_highest)

In [ ]:
# Query 2 : Currencies with rate > 1

df_gt1 = pd.read_sql("""
    Select target_currency, rate
    From exchange_rates
    Where base_currency = 'USD' AND rate > 1
    Order By rate DESC
    
""",conn)
print("\n Exchange Rate > 1: ")
print(df_gt1.head())

In [ ]:
 # Query 3 : Compare USD to EUR, GBP, INR, JPY

df_compare = pd.read_sql("""
    Select target_currency, rate
    From exchange_rates
    Where base_currency = 'USD' AND target_currency IN ('EUR', 'GBP', 'INR', 'JPY')
""",conn)

print("\n USD Compared to Major Currencies: ")
print(df_compare)

In [ ]:
# Save Query Results

output_path = "usd_vs_major_currencies.csv"
df_compare.to_csv(output_path, index = False)
print(f'\n Comparison saved to : {output_path}')

cursor.close()
conn.close()